In [0]:
from pyspark.sql.functions import to_timestamp,hour,minute,when,col,current_timestamp,date_format,lit,unix_timestamp,expr,abs,to_date,rank,datediff
from pyspark.sql.window import Window
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder,StringIndexer,VectorAssembler
from pyspark.ml.regression import LinearRegression,RandomForestRegressor

In [0]:
display(raw_df)

DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,Fault_Flag,TIME,DATE
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,1IF53ai7Xc0U56Y,53.5,5.1625,0.0,6183645.0,0,06:00:00,2020-05-15
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,7JYdWkrLSPkdwr4,58.42857143,5.628571429,0.0,7602960.0,0,06:00:00,2020-05-15
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,VHMLBKoKgIrUVDU,0.0,0.0,0.0,7206408.0,0,06:00:00,2020-05-15
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,WRmjgnKYAwPKWDb,43.85714286,4.228571429,0.0,7028673.0,0,06:00:00,2020-05-15
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,ZnxXDlPa8U1GXgE,58.57142857,5.671428571,0.0,6522172.0,0,06:00:00,2020-05-15
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,ZoEaEvLYb1n2sOq,24.5,2.366666667,0.0,7098099.0,0,06:00:00,2020-05-15
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,adLQvlD726eNBSB,56.125,5.425,0.0,6271355.0,0,06:00:00,2020-05-15
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,bvBOhCH3iADSZry,37.0,3.571428571,0.0,6316803.0,0,06:00:00,2020-05-15
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,iCRJl6heRkivqQ3,41.85714286,4.028571429,0.0,7177992.0,0,06:00:00,2020-05-15
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,ih0vzX44oOqAx2f,50.16666667,4.85,0.0,6185184.0,0,06:00:00,2020-05-15


In [0]:
raw_df = spark.table('solarpowerData.RUL_Dataset')
raw_df = raw_df.withColumn('TIME',date_format(col('DATE_TIME'),'HH:mm:ss'))
raw_df = raw_df.filter(col('TIME').between('06:00:00','18:15:00'))

In [0]:
raw_df = raw_df.withColumn('DATE',to_date(col('DATE_TIME')))

In [0]:
only_fault_df = raw_df.select('DATE_TIME','FAULT_FLAG','SOURCE_KEY','DATE').filter(col('Fault_Flag')==1)

In [0]:
display(only_fault_df)

DATE_TIME,FAULT_FLAG,SOURCE_KEY,DATE
2020-05-15T09:15:00.000+0000,1,zVJPv84UY57bAof,2020-05-15
2020-05-15T09:30:00.000+0000,1,zVJPv84UY57bAof,2020-05-15
2020-05-15T10:00:00.000+0000,1,1BY6WEcLGh8j5v7,2020-05-15
2020-05-15T10:00:00.000+0000,1,bvBOhCH3iADSZry,2020-05-15
2020-05-15T15:00:00.000+0000,1,z9Y9gH1T5YWrNuG,2020-05-15
2020-05-17T11:15:00.000+0000,1,1BY6WEcLGh8j5v7,2020-05-17
2020-05-17T11:45:00.000+0000,1,uHbuxQJl8lW7ozc,2020-05-17
2020-05-21T10:00:00.000+0000,1,1BY6WEcLGh8j5v7,2020-05-21
2020-05-21T10:00:00.000+0000,1,1IF53ai7Xc0U56Y,2020-05-21
2020-05-21T10:00:00.000+0000,1,7JYdWkrLSPkdwr4,2020-05-21


In [0]:
# rank_window = Window.partitionBy(only_fault_df['SOURCE_KEY'],only_fault_df['DATE']).orderBy(only_fault_df['DATE_TIME'])
# fault_rank_on_day = rank().over(rank_window)

In [0]:
fault_rank_df = only_fault_df.select(col('SOURCE_KEY').alias('FAULT_SOURCE_KEY'),col('DATE_TIME').alias('FAULT_DATE_TIME'))

In [0]:
rul_oncondition = (fault_rank_df['FAULT_DATE_TIME']>=raw_df['DATE_TIME']) & (fault_rank_df['FAULT_SOURCE_KEY'] == raw_df['SOURCE_KEY'])

In [0]:
rul_df = fault_rank_df.join(raw_df,on = rul_oncondition,how = 'inner')

In [0]:
display(rul_df)

FAULT_SOURCE_KEY,FAULT_DATE_TIME,DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,Fault_Flag,TIME,DATE
1IF53ai7Xc0U56Y,2020-05-31T11:00:00.000+0000,2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,1IF53ai7Xc0U56Y,53.5,5.1625,0.0,6183645.0,0,06:00:00,2020-05-15
1IF53ai7Xc0U56Y,2020-06-01T11:15:00.000+0000,2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,1IF53ai7Xc0U56Y,53.5,5.1625,0.0,6183645.0,0,06:00:00,2020-05-15
1IF53ai7Xc0U56Y,2020-05-26T14:00:00.000+0000,2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,1IF53ai7Xc0U56Y,53.5,5.1625,0.0,6183645.0,0,06:00:00,2020-05-15
1IF53ai7Xc0U56Y,2020-05-21T10:00:00.000+0000,2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,1IF53ai7Xc0U56Y,53.5,5.1625,0.0,6183645.0,0,06:00:00,2020-05-15
7JYdWkrLSPkdwr4,2020-06-06T11:30:00.000+0000,2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,7JYdWkrLSPkdwr4,58.42857143,5.628571429,0.0,7602960.0,0,06:00:00,2020-05-15
7JYdWkrLSPkdwr4,2020-06-01T11:15:00.000+0000,2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,7JYdWkrLSPkdwr4,58.42857143,5.628571429,0.0,7602960.0,0,06:00:00,2020-05-15
7JYdWkrLSPkdwr4,2020-05-24T14:00:00.000+0000,2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,7JYdWkrLSPkdwr4,58.42857143,5.628571429,0.0,7602960.0,0,06:00:00,2020-05-15
7JYdWkrLSPkdwr4,2020-05-21T10:00:00.000+0000,2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,7JYdWkrLSPkdwr4,58.42857143,5.628571429,0.0,7602960.0,0,06:00:00,2020-05-15
VHMLBKoKgIrUVDU,2020-06-06T12:45:00.000+0000,2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,VHMLBKoKgIrUVDU,0.0,0.0,0.0,7206408.0,0,06:00:00,2020-05-15
VHMLBKoKgIrUVDU,2020-05-21T10:00:00.000+0000,2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,VHMLBKoKgIrUVDU,0.0,0.0,0.0,7206408.0,0,06:00:00,2020-05-15


In [0]:
rul_df = rul_df.withColumn('RUL',(col('FAULT_DATE_TIME').cast('long') - col('DATE_TIME').cast('long'))/60)

In [0]:
display(rul_df)

FAULT_SOURCE_KEY,FAULT_DATE_TIME,DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,Fault_Flag,TIME,DATE,RUL
1IF53ai7Xc0U56Y,2020-05-31T11:00:00.000+0000,2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,1IF53ai7Xc0U56Y,53.5,5.1625,0.0,6183645.0,0,06:00:00,2020-05-15,23340.0
1IF53ai7Xc0U56Y,2020-06-01T11:15:00.000+0000,2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,1IF53ai7Xc0U56Y,53.5,5.1625,0.0,6183645.0,0,06:00:00,2020-05-15,24795.0
1IF53ai7Xc0U56Y,2020-05-26T14:00:00.000+0000,2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,1IF53ai7Xc0U56Y,53.5,5.1625,0.0,6183645.0,0,06:00:00,2020-05-15,16320.0
1IF53ai7Xc0U56Y,2020-05-21T10:00:00.000+0000,2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,1IF53ai7Xc0U56Y,53.5,5.1625,0.0,6183645.0,0,06:00:00,2020-05-15,8880.0
7JYdWkrLSPkdwr4,2020-06-06T11:30:00.000+0000,2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,7JYdWkrLSPkdwr4,58.42857143,5.628571429,0.0,7602960.0,0,06:00:00,2020-05-15,32010.0
7JYdWkrLSPkdwr4,2020-06-01T11:15:00.000+0000,2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,7JYdWkrLSPkdwr4,58.42857143,5.628571429,0.0,7602960.0,0,06:00:00,2020-05-15,24795.0
7JYdWkrLSPkdwr4,2020-05-24T14:00:00.000+0000,2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,7JYdWkrLSPkdwr4,58.42857143,5.628571429,0.0,7602960.0,0,06:00:00,2020-05-15,13440.0
7JYdWkrLSPkdwr4,2020-05-21T10:00:00.000+0000,2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,7JYdWkrLSPkdwr4,58.42857143,5.628571429,0.0,7602960.0,0,06:00:00,2020-05-15,8880.0
VHMLBKoKgIrUVDU,2020-06-06T12:45:00.000+0000,2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,VHMLBKoKgIrUVDU,0.0,0.0,0.0,7206408.0,0,06:00:00,2020-05-15,32085.0
VHMLBKoKgIrUVDU,2020-05-21T10:00:00.000+0000,2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,VHMLBKoKgIrUVDU,0.0,0.0,0.0,7206408.0,0,06:00:00,2020-05-15,8880.0


In [0]:
rul_df = rul_df.groupby('SOURCE_KEY','DATE_TIME','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION','PLANT_ID','AC_POWER','DC_POWER','DAILY_YIELD','TOTAL_YIELD').min('RUL')
rul_df = rul_df.withColumnRenamed('min(RUL)','RUL')
rul_df = rul_df.filter(col('RUL')>=1440)

In [0]:

str_ind = StringIndexer().setInputCol('SOURCE_KEY').setOutputCol('SOURCE_KEY_NUM')
ohe = OneHotEncoder().setInputCol('SOURCE_KEY_NUM').setOutputCol('encoded_Source_Key')

In [0]:
index_stages = [str_ind,ohe]
ohe_pipeline = Pipeline(stages = index_stages)
rul_indexed_df = ohe_pipeline.fit(rul_df).transform(rul_df)

In [0]:
independent_features = ['AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION','AC_POWER','DC_POWER','DAILY_YIELD','encoded_Source_Key']
model_input_vector = VectorAssembler().setInputCols(independent_features).setOutputCol('features')

rul_indexed_df = rul_indexed_df.withColumnRenamed('RUL','label')

In [0]:
rul_indexed_train_df,rul_indexed_test_df = rul_indexed_df.randomSplit([0.8,0.2])

In [0]:
lr_model = LinearRegression().setFeaturesCol('features').setLabelCol('label')
lr_model.setPredictionCol('LR_prediction')
rf_model = RandomForestRegressor().setFeaturesCol('features').setLabelCol('label')
rf_model.setPredictionCol('RF_prediction')


Out[269]: RandomForestRegressor_060c1ac2ae08

In [0]:
model_training_stages  = [model_input_vector]
rul_pipeline = Pipeline(stages=model_training_stages)
rul_trained_df = rul_pipeline.fit(rul_indexed_train_df).transform(rul_indexed_train_df)

In [0]:
rul_trained_df

Out[271]: DataFrame[SOURCE_KEY: string, DATE_TIME: timestamp, AMBIENT_TEMPERATURE: double, MODULE_TEMPERATURE: double, IRRADIATION: double, PLANT_ID: int, AC_POWER: double, DC_POWER: double, DAILY_YIELD: double, TOTAL_YIELD: double, label: double, SOURCE_KEY_NUM: double, encoded_Source_Key: vector, features: vector]

In [0]:


rul_lr_model = lr_model.fit(rul_trained_df)
rul_trained_df = rul_lr_model.transform(rul_trained_df)
print('Linear Regression Error: ' ,str(rul_lr_model.summary.meanAbsoluteError))

Linear Regression Error: 2889.5102373696313

In [0]:
rul_rf_model = rf_model.fit(rul_trained_df)
rul_trained_df = rul_rf_model.transform(rul_trained_df)
# print('Randomforest Regression Error: ' ,str(rul_rf_model.stages[-1].summary.meanAbsoluteError))

In [0]:
for i in zip(independent_features,rul_rf_model.featureImportances):
  print(i)

('AMBIENT_TEMPERATURE', 0.0783761701968495)
('MODULE_TEMPERATURE', 0.014006203740171076)
('IRRADIATION', 0.00817910636978636)
('AC_POWER', 0.009755507582849675)
('DC_POWER', 0.005613963107815661)
('DAILY_YIELD', 0.02009613127145579)
('encoded_Source_Key', 0.05763364032261603)

In [0]:
display(rul_trained_df.select('SOURCE_KEY','DATE_TIME','RF_Prediction','LR_Prediction','label'))

SOURCE_KEY,DATE_TIME,RF_Prediction,LR_Prediction,label
1BY6WEcLGh8j5v7,2020-05-15T12:00:00.000+0000,4457.771216520884,5168.2831724720145,2835.0
1BY6WEcLGh8j5v7,2020-05-15T12:15:00.000+0000,4504.370097179866,5092.413321179891,2820.0
1BY6WEcLGh8j5v7,2020-05-15T12:30:00.000+0000,4541.798143398354,5427.253235385118,2805.0
1BY6WEcLGh8j5v7,2020-05-15T12:45:00.000+0000,4754.672249749394,5774.794249933719,2790.0
1BY6WEcLGh8j5v7,2020-05-15T13:00:00.000+0000,4744.824502364036,5906.506226426363,2775.0
1BY6WEcLGh8j5v7,2020-05-15T13:15:00.000+0000,4794.290431567576,5923.152005299444,2760.0
1BY6WEcLGh8j5v7,2020-05-15T13:30:00.000+0000,4794.290431567576,5742.554804656611,2745.0
1BY6WEcLGh8j5v7,2020-05-15T13:45:00.000+0000,4794.290431567576,5731.847121504183,2730.0
1BY6WEcLGh8j5v7,2020-05-15T14:00:00.000+0000,4676.426922646589,6026.15190574882,2715.0
1BY6WEcLGh8j5v7,2020-05-15T14:15:00.000+0000,4676.426922646589,5977.390852302167,2700.0
